In [1]:
import json
import os
import pandas
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

/Users/victoria/GitHub/venvs/bespoke/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
metrc_download_summary_companies_query = create_queries.create_metrc_download_summary_companies_query()
metrc_download_summary_companies_dataframe = pandas.read_sql_query(metrc_download_summary_companies_query, engine)

In [3]:
company_records = metrc_download_summary_companies_dataframe.to_dict('records')

In [8]:
company_records

[{'name': 'Aureum Labs',
  'identifier': 'AL',
  'id': 'ddc07d90-aac6-4af0-ae2e-c7b9d0ad9c59'},
 {'name': 'Bella Sol',
  'identifier': 'BS',
  'id': '278bd48f-dc47-44c9-9c72-bba14521a9c8'},
 {'name': 'Big Medicine Cannabissary',
  'identifier': 'BMC',
  'id': '8ebb73d5-91a7-4f4f-b913-b0e67d3d4f3f'},
 {'name': 'Big Tree Industries',
  'identifier': 'BT',
  'id': 'bfd3e6e7-d697-4c9d-8770-ec6ef63e34fd'},
 {'name': 'Boston Bud Factory',
  'identifier': 'BBF',
  'id': '08ef04eb-a9b0-45a3-aee6-8d0b131a8eab'},
 {'name': 'Buddies',
  'identifier': 'BD',
  'id': 'fe9cb9e0-56e3-4d29-87f8-cb01b4b93d55'},
 {'name': 'CHO',
  'identifier': 'CHO',
  'id': 'a3e69e1a-25e8-49d3-be31-6a1c1db1f17f'},
 {'name': 'Cali Greens LLC',
  'identifier': 'CG',
  'id': '7c3a4bb6-9905-4fb1-8d45-23e49049e8ba'},
 {'name': 'California Patients Alliance',
  'identifier': 'CPA',
  'id': '6472346a-7896-495d-a342-7ff1ac88ea5a'},
 {'name': 'California Patients Club',
  'identifier': 'CPC',
  'id': 'b9b51899-74bc-4577-886e-c1

In [4]:
company_to_retailer_licenses = {}
company_identifier_to_us_state = {}

for company_record in company_records:
    company_identifier = company_record['identifier']
    company_name = company_record['name']

    key = (company_identifier, company_name)
    
    print(f'Checking if company {company_name} ({company_identifier}) holds any retailer licenses...')
    company_licenses_query = create_queries.create_company_licenses_query(company_identifier)
    company_licenses_dataframe = pandas.read_sql_query(company_licenses_query, engine)
    company_license_records = company_licenses_dataframe.to_dict('records')
    for company_license_record in company_license_records:
        us_state = company_license_record['us_state']
        company_identifier_to_us_state[company_identifier] = us_state

        license_number = company_license_record['license_number']
        license_category = company_license_record['license_category']
        if license_category in ['Multiple', 'Retailer']:
            if key not in company_to_retailer_licenses:
                company_to_retailer_licenses[key] = []
            company_to_retailer_licenses[key].append(license_number)

    print(f'Checking if company {company_name} ({company_identifier}) has any sales receipts...')
    company_count_metrc_sales_receipts_query = create_queries.create_company_count_metrc_sales_receipts_query(company_identifier)
    company_count_metrc_sales_receipts_dataframe = pandas.read_sql_query(company_count_metrc_sales_receipts_query, engine)
    
    has_sales_receipts = len(company_count_metrc_sales_receipts_dataframe.index) > 0
    if has_sales_receipts:
        company_count_metrc_sales_receipt_record = company_count_metrc_sales_receipts_dataframe.to_dict('records')[0]
        sales_receipts_count = company_count_metrc_sales_receipt_record['count']
        if key not in company_to_retailer_licenses:
            print(f'WARNING!!!')
            print(f'WARNING: found company {company_name} ({company_identifier}) with {sales_receipts_count} sales receipts but no retailer license!')
    else:
        if key in company_to_retailer_licenses:
            print(f'WARNING!!!')
            print(f'WARNING: found company {company_name} ({company_identifier}) with 0 sales receipts but with retailer license(s)!')
            del company_to_retailer_licenses[key]

Checking if company Aureum Labs (AL) holds any retailer licenses...
Checking if company Aureum Labs (AL) has any sales receipts...
Checking if company Bella Sol (BS) holds any retailer licenses...
Checking if company Bella Sol (BS) has any sales receipts...
Checking if company Big Medicine Cannabissary (BMC) holds any retailer licenses...
Checking if company Big Medicine Cannabissary (BMC) has any sales receipts...
Checking if company Big Tree Industries (BT) holds any retailer licenses...
Checking if company Big Tree Industries (BT) has any sales receipts...
Checking if company Boston Bud Factory (BBF) holds any retailer licenses...
Checking if company Boston Bud Factory (BBF) has any sales receipts...
Checking if company Buddies (BD) holds any retailer licenses...
Checking if company Buddies (BD) has any sales receipts...
Checking if company CHO (CHO) holds any retailer licenses...
Checking if company CHO (CHO) has any sales receipts...
Checking if company Cali Greens LLC (CG) holds 

Checking if company PG Manufacturing (PGM) has any sales receipts...
Checking if company People's Remedy (PRP) holds any retailer licenses...
Checking if company People's Remedy (PRP) has any sales receipts...
Checking if company People's Remedy Lone Palm (PRL) holds any retailer licenses...
Checking if company People's Remedy Lone Palm (PRL) has any sales receipts...
Checking if company Purple Lotus (PL) holds any retailer licenses...
Checking if company Purple Lotus (PL) has any sales receipts...
Checking if company Quality Roots (QR) holds any retailer licenses...
Checking if company Quality Roots (QR) has any sales receipts...
Checking if company Royal Apothecary (RA) holds any retailer licenses...
Checking if company Royal Apothecary (RA) has any sales receipts...
Checking if company Royal Greens Delivery (RGD) holds any retailer licenses...
Checking if company Royal Greens Delivery (RGD) has any sales receipts...
Checking if company San Fernando Valley Patients Cooperative (SFV) 

In [ ]:
# for company_tuple, retailer_licenses in company_to_retailer_licenses.items():
#     company_identifier, company_name = company_tuple
#     print(f'Company {company_name} ({company_identifier}) holds Retailer licenses: {retailer_licenses}')

In [5]:
len(company_to_retailer_licenses.keys())

63

In [6]:
# Output to copy to XLSX
# Columns: company name, company identifier, US state, semi-colon separated license numbers
retailer_company_tuples = []

for company_tuple, retailer_licenses in company_to_retailer_licenses.items():
    company_identifier, company_name = company_tuple
    company_licenses = ";".join(retailer_licenses)
    us_state = company_identifier_to_us_state[company_identifier]
#     print((company_name, company_identifier, us_state, retailer_licenses))
    retailer_company_tuples.append((company_name, company_identifier, us_state, company_licenses))

In [7]:
columns = [
    'Company name',
    'Company identifier',
    'US state',
    'Company licenses',
]
retailer_companies_dataframe = pandas.DataFrame(retailer_company_tuples, columns=columns)
retailer_companies_dataframe

,Company name,Company identifier,US state,Company licenses
0,Bella Sol,BS,MI,PC-000185;AU-R-000156
1,Big Medicine Cannabissary,BMC,CO,402-00930
2,Boston Bud Factory,BBF,MA,MR281525
3,CHO,CHO,MI,AU-R-000575
4,Cali Greens LLC,CG,CA,C9-0000157-LIC
...,...,...,...,...
58,Universal Herbal Health Center,UHHC,CA,C9-0000082-LIC;C10-0000817-LIC
59,Uprooted (San Ysidro),UR,CA,C10-0000776-LIC;C10-0000246-LIC;C10-0000464-LI...
60,Verts,VS,CO,402-00840;402R-00545
61,Voyage,VOY,CA,C10-0000802-LIC;C12-0000030-LIC;C12-0000159-LIC


In [11]:
retailer_companies_dataframe[retailer_companies_dataframe['US state'] == 'CA']

,Company name,Company identifier,US state,Company licenses
4,Cali Greens LLC,CG,CA,C9-0000157-LIC
5,California Patients Alliance,CPA,CA,C9-0000348-LIC;C10-0000461-LIC
6,California Patients Club,CPC,CA,C9-0000056-LIC
7,California Street Cannabis,CSC,CA,C10-0000670-LIC;C10-0000918-LIC
8,California's Choice Collective,CCC,CA,C12-0000087-LIC
9,Calm and Collective,CC,CA,C10-0000758-LIC
13,Dubs Green Garden,DGG,CA,C9-0000016-LIC
15,E-Leaf,EL,CA,C9-0000370-LIC
16,Embarc Alameda,EMA,CA,C10-0000774-LIC
17,Embarc Fairfield,EMF,CA,C10-0000939-LIC


In [9]:
import time
from datetime import date

TODAY_DATE = date.today().strftime('%m-%d-%Y')
NOW = int(time.time())

retailer_companies_file_name = f'~/Downloads/retailer_companies_file_name_{TODAY_DATE}_{NOW}'

num_retailer_companies = len(retailer_companies_dataframe.index)
print(f'Exporting {num_retailer_companies} retailer companies to files...')
retailer_companies_dataframe.reset_index().to_excel(f'{retailer_companies_file_name}.xlsx')
# export_incoming_transfer_packages_dataframe.reset_index().to_csv(f'{incoming_transfer_packages_file_name}.csv')
print(f'Exported {num_retailer_companies} retailer companies to files')

Exporting 63 retailer companies to files...
Exported 63 retailer companies to files
